# Quality assessment of satellite sea ice thickness

## Import libraries

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

plt.style.use("seaborn-v0_8-notebook")

## Set parameters

In [ ]:
year_start_envisat = 2002
year_stop_envisat = 2010

year_start_cryosat = 2010
year_stop_cryosat = 2020

## Define request

In [ ]:
collection_id = "satellite-sea-ice-thickness"

year_ranges = {
    "envisat": range(year_start_envisat, year_stop_envisat + 1),
    "cryosat_2": range(year_start_cryosat, year_stop_cryosat + 1),
}

requests = {}
for satellite, year_range in year_ranges.items():
    requests[satellite] = {
        "satellite": satellite,
        "version": "2_0",
        "cdr_type": "cdr",
        "variable": "all",
        "year": list(map(str, year_range)),
        "month": [f"{month:02d}" for month in [1, 2, 3, 4, 10, 11, 12]],
    }

chunks = {"year": 1}

## Define function to cache

In [ ]:
def sea_ice_thickness_linear_trend(ds, **kwargs):
    da = ds["sea_ice_thickness"]
    if not kwargs.get("p_value") and not kwargs.get("rmse"):
        return diagnostics.time_weighted_linear_trend(da, **kwargs)
    return xr.Dataset(
        diagnostics.time_weighted_linear_trend(
            da.chunk(time=-1),
            **kwargs,
        )
    )


def monthly_weighted_linear_trend(ds, **kwargs):
    return ds.groupby("time.month").apply(sea_ice_thickness_linear_trend, **kwargs)

## Download and transform

In [ ]:
funcs = (diagnostics.monthly_weighted_mean, diagnostics.monthly_weighted_std)
objects = []
for satellite, request in requests.items():
    print(f"{satellite=}")
    for func in funcs:
        _, name = func.__name__.rsplit("_", 1)
        ds = download.download_and_transform(
            collection_id,
            request,
            chunks=chunks,
            transform_chunks=False,
            transform_func=func,
            transform_func_kwargs={"weights": False},
        )
        da = ds["sea_ice_thickness"].rename(name)
        da.attrs["long_name"] = f"{name.title()} of " + da.attrs["long_name"]
        objects.append(da.expand_dims(satellite=[satellite]))

    ds = download.download_and_transform(
        collection_id,
        request,
        chunks=chunks,
        transform_chunks=False,
        transform_func=monthly_weighted_linear_trend,
        transform_func_kwargs={"weights": False, "p_value": True, "r2": True},
    )
    objects.append(ds.expand_dims(satellite=[satellite]))
ds = xr.merge(objects)

## Plot maps

In [ ]:
ds = ds.where(ds["mean"].notnull().compute(), drop=True)
for var, da in ds.data_vars.items():
    facet = plot.projected_map(
        da,
        projection=ccrs.Stereographic(central_latitude=90),
        robust=True,
        col="satellite",
        row="month",
    )
    plt.show()